# Django 7일차

### 인증과 권한

HTTP의 특징
1. 비 연결 지향
2. 무상태

서버-클라이언트간 '상태'를 유지하기 위한 기술
1. 쿠키
2. 세션

로그인 : 사용자에 대한 세션을 생성하는 것

로그아웃 : 세션을 클라이언트와 서버에서 삭제하는 것

### Authentication with User

User Object 와 User CRUD에 대한 이해
- 회원 가입, 회원 탈퇴, 회원정보 수정, 비밀번호 변경

회원가입
- User를 Create 하는 것이며 UserCreationForm built-in form을 사용

UserCreationForm
- 주어진 username과 password로 권한이 없는 새 user를 생성하는 ModelForm
- 3개의 필드를 가짐
    1. username(from the user model)
    2. password1
    3. password2

회원 가입 페이지 작성
```python
    # accounts/urls.py
    app_name = 'accounts'
    urlpatterns = [
        ...,
        path('signup/', views.signup, name='signup')
    ]

```


```html
    <!-- accounts/signup.html -->
    {% extends "base.html" %}

    {% block content %}
        <h1>회원가입</h1>
        
        <form action="{% url 'accounts:signup' %}" method="POST">
            {% csrf_token %}
            {{form.as_p}}
            <input type="submit" value="회원가입">
        </form>
        <a href="{% url 'articles:index' %}">목록보기</a>
    {% endblock content %}
```

```python
    # accounts/views.py
    def signup(request):
        if request.method=='POST':
            form = UserCreationForm(request.POST)
            if form.is_valid():
                return redirect('articles:index')
        else:
            form = UserCreationForm()
        context = {'form':form}
        return render(request, 'accounts/signup.html', context)
```

회원가입 링크 작성 후 페이지 확인
```html
    <!-- base.html -->
    <a href="{% url 'accounts:signup' %}">회원가입</a>
```

회원가입 진행 후 에러 페이지 확인
- UserCreationForm이 우리가 대체한 커스텀 유저 모델이 아닌 기존 유저 모델로 인해 작성된 클래스 이기 때문에 처리해줘야함

Custom user & Built-in auth forms
- Custom user와 기존 Built-in auth forms 간의 관계
- Custom user로 인한 Built-in auth forms 변경

AbstractBaseUser의 모든 subclass와 호환되는 forms
- 아래 Form 클래스는 User 모델을 대체하더라도 커스텀 하지 않아도 사용 가능
- 기존 User 모델을 참조하는 Form이 아님
    1. AuthenticationForm
    2. SetPasswordForm
    3. PasswordChangeForm
    4. AdminPasswordChangeForm

커스텀 유저 모델을 사용하려면 다시 작성하거나 확장해야 하는 forms
1. UserCreationForm
2. UserChangeForm
- 두 form 모두 class Meta: model = User가 등록된 form 이기 떄문에 반드시 커스텀(확장)해야 함
```python
class CustomUserCreationForm(UserCreationForm):
    
    class Meta(UserCreationForm.Meta):
        #accounts.User
        model = get_user_model()

class CustomUserChangeForm(UserChangeForm):
    
    class Meta(UserChangeForm.Meta):
        model = get_user_model()

```

get_user_model()
- `현재 프로젝트에서 활성화된 사용자 모델(active user model)`을 반환
- 직접 참조하지 않는 이유
    - 예를 들어 기존 User 모델이 아닌 User 모델을 커스텀 한 상황에서는 커스텀 User 모델을 자동으로 반환해주기 떄문
- Django는 User 클래스를 직접 참조하는 대신 `get_user_model()`을 사용해 참조해야 한다고 강조하고 있음

CustomUserCreateionForm()으로 대체하기

```python
    def signup(request):
        if request.method=='POST':
            form = CustomUserCreationForm(request.POST)
            if form.is_valid():
                user = form.save()
                return redirect('articles:index')
        else:
            form = UserCreationForm()
        context = {'form':form}
        return render(request, 'accounts/signup.html', context)
```

회원 가입 후 곧바로 로그인 진행

`UserCreationForm`의 `save()`메서드 : user를 반환함

```python
    def signup(request):
        if request.method=='POST':
            form = CustomUserCreationForm(request.POST)
            if form.is_valid():
                user = form.save()
                # 로그인 추가
                auth_login(request, user)
                return redirect('articles:index')
        else:
            form = UserCreationForm()
        context = {'form':form}
        return render(request, 'accounts/signup.html', context)
```

### 회원 탈퇴

- 회원 탈퇴 하는 것은 DB에서 유저를 DELETE 하는 것과 같음 -> POST

회원 탈퇴 로직 작성

```python
# accounts/urls.py
app_name = 'accounts'
urlpatterns = [
    path('delete/', views.delete, name='delete'),
]
```

```python
# accounts/views.py
def delete(request):
    request.user.delete()
    return redirect('articles:index')
```

```html
<!-- base.html -->
<form action="{% url 'accounts:delete' %}" method="POST">
    {% csrf_token %}
    <input type="submit" value="회원탈퇴">
</form>
```


탈퇴하면서 해당 유저의 세션 정보도 함께 지우고 싶을 경우

- 탈퇴(1) 후 로그아웃(2) 의 순서가 바뀌면 안됨
- 먼저 로그아웃 해버리면 해당 요청 객체 정보가 없어지기 때문에 탈퇴에 필요한 정보 또한 없어지기 때문

```python
# accounts/views.py
def delete(request):
    request.user.delete()
    # auth_logout(request)
    return redirect('articles:index')
```

회원정보 수정
- 회원정보 수정은 User를 Update 하는 것이며 UserChangeForm built-in form을 사용

UserChangeForm
- 사용자의 정보 및 권한을 변경하기 위해 admin 인터페이스에서 사용되는 ModelForm
- UserChangeForm 또한 ModelForm 이기 때문에 instance 인자로 기존 user 데이터 정보를 받는 구조 또한 동일함
- 이미 이전에 `CustomUserChangeForm`으로 확장했기 때문에 `CustomUserChangeForm`을 사용

회원정보 수정 로직 작성

```python
# accounts/urls.py
app_name = 'accounts'
urlpatterns = [
    path('update/', views.update, name='update'),
]
```

```python
# accounts/views.py
def update(request):
    if request.method=='POST':
        pass
    else:
        form = CustomUserChangeForm(instance=request.user)
    context = {
        'form':form,
    }
    return render(request, 'accounts/update.html', context)
```

```html
<!-- accounts/update.html -->
{% extends "base.html" %}

{% block content %}
    <h1>회원정보 수정</h1>
    
    <form action="{% url 'accounts:update' %}" method="POST">
        {% csrf_token %}
        {{form.as_p}}
        <input type="submit" value="수정하기">
    </form>
    <a href="{% url 'articles:index' %}">목록보기</a>
{% endblock content %}
```

```html
<!-- base.html -->
<a href="{% url 'accounts:update' %}">정보수정</a>  
```


UserChangeForm 사용 시 문제점
- 일반 사용자가 접근해서는 안 될 정보들(fields)까지 모두 수정이 가능해짐
    - admin 인터페이스에서 사용되는 ModelForm이기 때문
- 따라서 UserChangeForm을 상속받아 작성해 두었던 서브 클래스 CustomUserForm에서 접근 가능한 필드를 조정해야 함

